In [1]:
from symtable import Symbol
from StockStageEstimator import StockStageEstimator
import pandas as pd
import numpy as np
import random
from datetime import datetime
from stage import fullPrint
# import mariadb
import sys
import os
# from dotenv import load_dotenv

# load_dotenv()

### ENV CONSTANT
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_USER = os.getenv('DB_USER')

### Connect To DB
# try:
#     conn = mariadb.connect(
#         user=DB_USER,
#         password=DB_PASSWORD,
#         host=DB_HOST,
#         port=3306,
#         database="stock",
#         autocommit=True
#     )
# except mariadb.Error as e:
#     print(f"Error connecting to MariaDB Platform: {e}")
#     sys.exit(1)
# cur = conn.cursor()

### CONSTANT VARIABLES ###

## Mutation Noise 
mutationNoises = []
mutationNoises.append(np.random.normal(0, 0.025)) # 0 - Five Year High
mutationNoises.append(np.random.normal(0, 0.075)) # 1 - Volume Perc
mutationNoises.append(np.random.normal(0, 0.015)) # 2 - RS
mutationNoises.append(np.random.normal(0, 0.005)) # 3 - Slope
mutationNoises.append(np.random.normal(0, 0.0125)) # 4 - Price v WMA30
mutationNoises.append(np.random.normal(0, 0.015)) # 5 - Price v Initial Support
mutationNoises.append(np.random.normal(0, 0.005)) # 6 - Prev Trough v Peak
mutationNoises.append(np.random.normal(0, 0.01)) # 7 - Price v Prev Support
mutationNoises.append(np.random.normal(0, 0.0125)) # 8 - Price v Prev Close
mutationNoises.append(np.random.normal(0, 0.0005)) # 9 - Sector MA30 Slope
mutationNoises.append(np.random.normal(0, 0.0025)) # 10 - Sector RS
mutationNoises.append(np.random.normal(0, 0.0025)) # 11 - SPDF Price v WMA30
mutationNoises.append(np.random.normal(0, 0.0002)) # 12 - SPDF WMA30 Slope
mutationNoises.append(np.random.normal(0, 0.0025)) # 13 - SPDF Sell P v WMA30
mutationNoises.append(np.random.normal(0, 0.0002)) # 14 - SPDF Sell WMA30 Slope


In [2]:
## Get returns of a ticker list
def calculateGroupReturn(list):
    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for industry in list:
        print(industry)
        symbol = industry[1]
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/industriesData/"+industry[0]+'/'+symbol+".pkl")
        except:
            print("error "+ symbol)
            continue
        if not (df['Close'] != 0).all() or df.empty:
            if df.empty:
                print("error "+ symbol)
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = monthlyReturn.index[0]
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                lastPrice = element[0]
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = lastPrice
                monthCheck = index.strftime("%Y:%m")
    return monthlyReturn

In [3]:
## Calculate Goodsector & Initialize Estimator Object 
def evalReturn(individual):
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    
    sectorDfList = pd.read_pickle('stockData/sectorDfList.pkl')
    goodSectorDf = sectorDfList[0].drop(['Open','High','Low','Close','Currency','MA30','MA30Slope','RS','sector','percent','Volume'],axis=1)
    goodSectorDf.index = sectorDfList[0].index
    goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
    for date in sectorDfList[0].index:
        listOfSector = []
        for df in sectorDfList:
            try:
                if df.at[date,'MA30Slope'] > 1 or df.loc[date,'RS'] > 0.1:
                    if df.at[date,'Close'] > df.at[date,'MA30']:
                        listOfSector.append(df.at[date,'sector'])
            except: 
                print(df.iloc[-1]['sector'])
        goodSectorDf.at[date,'Sectors'] = set(listOfSector)

    stockStageEstimator = StockStageEstimator(individual[0], goodSectorDf)
    scores = stockStageEstimator.score()
    debugFile = open("estimatorData/debug"+date.today().strftime('%Y-%m-%d')+".txt","a")
    debugFile.write(str(individual[0])+"\n")
    if scores == -1:
        debugFile.write(str(np.average(scores))+"\n")
        debugFile.write(str(stockStageEstimator.getReturns())+"\n")
        debugFile.close()
        return 0,
    result = stockStageEstimator.result()
    debugFile.write(str(np.average(scores[0]))+"\n")
    debugFile.write(str(np.average(scores[1]))+"\n")
    debugFile.write(str(stockStageEstimator.getReturns())+"\n")
    debugFile.write(str(result)+"\n")
    debugFile.close()
    if np.average(scores[0]) >  0.1:
        return 0,
    if np.average(scores[1]) > 1.3:
        return 0,
    paramFile = open("estimatorData/params"+date.today().strftime('%Y-%m-%d')+".txt","a")
    paramFile.write(str(individual[0])+"\n")
    paramFile.write(str(result)+"\n")
    paramFile.close()

    # cur.execute("INSERT INTO Params (param,result) VALUES (?, ?)", (individual[0], result))

    resultFile = open("estimatorData/resultML"+date.today().strftime('%Y-%m-%d')+".txt","a")
    resultFile.write(str(individual[0])+"\n")
    resultFile.write(str(np.average(scores[0]))+"\n")
    resultFile.write(str(np.average(scores[1]))+"\n")
    resultFile.write(str(stockStageEstimator.getReturns())+"\n")
    resultFile.write(str(result)+"\n")
    resultFile.close()
    return result,

In [4]:
##REMOVE LATER!!!
evalReturn([[0.14057942717262228, 1.9505086899600719, 0.32738233131096084, 0.9361257255250749, 1.0177603487709328, 1.1745242387323218, 0.8695933218754314, 1.0656706590763125, 1.3587720212104732, 0.9996158063478496, -0.07328054903044724, 0.8303717189888326, 0.9974598470770332, 0.8182778600022654, 0.9777705516269057]
])

[0.14057942717262228, 1.9505086899600719, 0.32738233131096084, 0.9361257255250749, 1.0177603487709328, 1.1745242387323218, 0.8695933218754314, 1.0656706590763125, 1.3587720212104732, 0.9996158063478496, -0.07328054903044724, 0.8303717189888326, 0.9974598470770332, 0.8182778600022654, 0.9777705516269057]
0.5
84.47472715243694
-0.39944163229812035


(0,)

In [ ]:
### DEAP Evolutionary Functions ###

## Generating individuals
def generate_random_num_attr():
  original = np.array([0,  # 0 - Five Year High
                        0, # 1 - Volume Perc
                        0, # 2 - RS
                        0, # 3 - Slope
                        0, # 4 - Price v WMA30
                        0, # 5 - Price v Initial Support
                        0, # 6 - Prev Trough v Peak
                        0, # 7 - Price v Prev Support
                        0, # 8 - Price v Prev Close
                        0.995, # 9 - Sector MA30 Slope
                        0, # 10 - Sector RS
                        0.9, # 11 - SPDF Price v WMA30
                        0.995, # 12 - SPDF WMA30 Slope
                        0.85, # 13 - SPDF Sell P v WMA30
                        0.985]) # 14 - SPDF Sell WMA30 Slope
  noises = []
  sum_list = []

In [ ]:
  noises.append(random.uniform(0,1))
  noises.append(random.uniform(0.1,2.5))
  noises.append(random.uniform(-0.1,0.4))
  noises.append(random.uniform(0.9,1.1))
  noises.append(random.uniform(1,1.4))
  noises.append(random.uniform(1, 1.3))
  noises.append(random.uniform(0.8, 1))
  noises.append(random.uniform(0.8, 1.1))
  noises.append(random.uniform(1.2, 1.5))
  noises.append(np.random.normal(0, 0.008))
  noises.append(np.random.normal(0, 0.2))

  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.008))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.008))

  while original[11] + noises[11] < original[13] + noises[13]:
    noises[11] = np.random.normal(0, 0.05)
    noises[13] = np.random.normal(0, 0.05)
  while original[12] + noises[12] < original[14] + noises[14]:
    noises[12] = np.random.normal(0, 0.004)
    noises[14] = np.random.normal(0, 0.004)
  
  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list

In [ ]:
## Mutating Individuals
def mutate(individual):
    size = len(individual)
    for i in range(0,size):
        if random.random() > 0.9:
            individual[i] = individual[i] + mutationNoises[i]
    return individual,